# Importing Dataset

In [ ]:
import pickle
chromosomeFile = "Data//matchesList.txt"
labelsFile = "Data//labelList.txt"

with open(chromosomeFile, 'rb') as f:
    chromosomeList = pickle.load(f)

with open(labelsFile, 'rb') as f:
    labels = pickle.load(f)

# Genetic Programming

1. Initialize random population
2. Evaluate each one
3. Select 2 good ones (tournament, roulette wheel, monte carlo)
4. Crossover between them
5. Mutate the solution (cant be names 0 or 8)
6. Evaluate
7. Replace if better than worst

In [ ]:
for x in chromosomeList:
    del x[8]
    del x[0]

In [ ]:
import random as r

### Create Population

In [ ]:
def createSolution():
    solution = [0]*len(chromosomeList[0])
    for i in range(len(solution)):
        solution[i] = r.uniform(-10,10)
    return solution

In [ ]:
def createPopulation(populationSize):
    population = []
    for i in range(0,populationSize):
        population.append(createSolution())
    return population

### Evaluate Solutions

In [ ]:
def createSolutionLabels(solution):
    localLabels = []
    for i in range(0,len(chromosomeList)):
        t1Val = 0
        t2Val = 0
        for j in range(0,len(solution)):
            if(j<=6):
                t1Val += solution[j]*chromosomeList[i][j]
            else:
                t2Val += solution[j]*chromosomeList[i][j]
        result = t1Val - t2Val
        if(result>500):
            localLabels.append("H")
        elif(result<-500):
            localLabels.append("A")
        elif(result<500 and result>-500):
            localLabels.append("D")
    return localLabels

                


In [ ]:
def evaluateSolution(solution):
    sLabels = createSolutionLabels(solution)
    correct = 0
    for i in range(0,len(sLabels)):
        if(sLabels[i]==labels[i]):
            correct += 1
    return correct/len(sLabels)

### Selection

In [ ]:
def tournamentSelection(population, tournamentSize):
    selected = []
    for i in range(0,tournamentSize):
        selected.append(population[r.randint(0,len(population)-1)])
    
    #Evaluate the selected solutions
    selectedEval = []
    for i in range(0,len(selected)):
        selectedEval.append(evaluateSolution(selected[i]))
    #Find the best solution
    best = selectedEval.index(max(selectedEval))
    return selected[best]

### Crossover

In [ ]:
def crossover(solution1, solution2, crossoverPoint):
    child = solution1[0:crossoverPoint] + solution2[crossoverPoint:]
    return list(child)

### Mutation

In [ ]:
def mutation(solution, mutationCount):
    for i in range(0,mutationCount):
        index = r.randint(0,len(solution)-1)
        solution[index] = r.uniform(-10,10)
    return solution

### Evolve Function

In [ ]:
def evolve(populationSize, tournamentSize, mutationCount, generations):
    population = createPopulation(populationSize)
    stages = [0]*generations
    for i in range(0,generations):
        fitness = [0]*len(population)
        for j in range(0,len(population)):
            fitness[j] = evaluateSolution(population[j])
        child1 = tournamentSelection(population, tournamentSize)
        child2 = tournamentSelection(population, tournamentSize)
        child = crossover(child1, child2, r.randint(0,len(child1)-1))
        child = mutation(child, mutationCount)
        newFitness = evaluateSolution(child)
        if(newFitness > min(fitness)):
            population[fitness.index(min(fitness))] = child
        stages[i] = [i, max(fitness), sum(fitness)/len(fitness), fitness.index(max(fitness))]
    text = "Population Size: " + str(populationSize) + ", Tournament Size: " + str(tournamentSize) + ", Mutation Count: " + str(mutationCount) + ", Generations: " + str(generations) + ", Best: " + str(max(fitness)) + ", Average: " + str(sum(fitness)/len(fitness))
    return [text, population[fitness.index(max(fitness))], stages]

# Usage

In [ ]:
from timeit import default_timer as timer
import datetime
def run(p, t, m, g):
    start = timer()
    populationSize = p
    tournamentSize = t
    mutationCount = m
    generations = g
    infoText, chromosome, stages = evolve(populationSize, tournamentSize, mutationCount, generations)
    infoText = infoText + ", Time: " + str(timer()-start)
    infoText = infoText + ", Date: " + str(datetime.datetime.now())

    with open("Results//GP//testsGP.txt", 'a') as f:
        f.write(str(infoText) + "\n")

    with open("Results//GP//chromosomesGP.txt", 'a') as f:
        f.write(str(chromosome) + "\n")
    
    date = str(datetime.datetime.now())
    date = date.replace(" ","")[5:len(date)-8]
    date = "test" + date + ".txt"
    date = date.replace(":","_")
    date = date[:9] + "_" + date[9:]
    date = date[:len(date)-4] + "-GP" + date[len(date)-4:]
    print(date)
    
    with open("Results//GP//Individual//" + str(date), 'w') as f:
        #f.write(str(stages))
        f.write("Generation, Best, Average, Index\n")
        for i in stages:
            f.write(str(i).replace("[","").replace("]","") + "\n")
    
    return "done"

In [ ]:
date = str(datetime.datetime.now())
date = date.replace(" ","")[5:len(date)-8]
date = "test" + date + ".txt"
date = date.replace(":","_")
date = date[:9] + "_" + date[9:]
print(date)
run(100,5,2,10)

In [ ]:
populationSizes = [10,50,100,500]
tournamentSizes = [3,6]
mutationCounts = [1,2,3]
genarations = [10,100,250,500,1000]
for g in genarations:
    for m in mutationCounts:
        for t in tournamentSizes:
            for p in populationSizes:
                #run(p, t, m, g)
                pass